In [ ]:
import os
import json
import joblib
import time
import pandas as pd
import numpy as np
import random
from IPython.display import display, HTML
import torch

# Better pandas table
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Using: ', device)

In [ ]:
!pip install -U sentence-transformers
!pip install bert-extractive-summarizer

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# path_models = '/content/drive/MyDrive/w266/models/'
# path_outputs = '/content/drive/MyDrive/w266/outputs/'
# path_processed_data= '/content/drive/MyDrive/w266/processed_data/'

In [ ]:
path_models = './models/'
path_outputs = './outputs/'
path_processed_data= './processed_data/'

In [ ]:
data_name = 'cnn_test'
input_filename = data_name + '_v3.json' 
#input_filename = data_name + '_summary_v2.json' 
#out_filename = data_name + '_summary_abs.json'
out_filename = data_name + '_summary_v3.json'

In [ ]:
jsonFile = open(os.path.join(path_processed_data, input_filename), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()


X = details_dict['text']
size = len(X)

Y = details_dict['headlines']
Z = details_dict['keywords']
E = details_dict['entities']
text_ner = details_dict['text_ner']
S1s = details_dict['sentence_1s']
S3s = details_dict['sentence_3s']
S1 = details_dict['summary_art']
S2 = details_dict['summary_ext']
S3 = details_dict['summary_abs']
S4 = details_dict['summary_extabs']
S5 = details_dict['summary_ner']
#S6 = details_dict['summary_t5']
S6 = ['']*size

P0 = details_dict['predict_1s']
P1 = details_dict['predict_3s']
P2 = details_dict['predict_text']
P3 = details_dict['predict_ext']
P4 = details_dict['predict_abs']
P5 = details_dict['predict_extabs']
P6 = details_dict['predict_ner']
P7 = details_dict['predict_t5']


details = {
    
    'text' : X,
    'headlines' : Y,
    'keywords' : Z,
    'summary_art' : S1,
    'entities' : E,
    'text_ner' : text_ner,
    'sentence_1s' : S1s,
    'sentence_3s' : S3s,
    'summary_ext' : S2,
    'summary_abs' : S3,
    'summary_extabs' : S4,
    'summary_ner' : S5,
    'summary_t5' : S6,
    'predict_1s' : P0,
    'predict_3s' : P1,
    'predict_text' : P2,
    'predict_ext' : P3,
    'predict_abs' : P4,
    'predict_extabs' : P5,
    'predict_ner' : P6,
    'predict_t5' : P7
}



# size = len(X)

# S2 = [""] * size
# S3 = [""] * size
# S4 = [""] * size
# S5 = [""] * size
# P1 = [""] * size
# P2 = [""] * size
# P3 = [""] * size
# P4 = [""] * size
# P5 = [""] * size

# details = {
#     'text' : X,
#     'headlines' : Y,
#     'keywords' : Z,
#     'summary_art' : S1,
#     'summary_ext' : S2,
#     'summary_abs' : S3,
#     'summary_extabs' : S4,
#     'summary_ner' : S5,
#     'predict_text' : P1,
#     'predict_ext' : P2,
#     'predict_abs' : P3,
#     'predict_extabs' : P4,
#     'predict_ner' : P5
# }

df_test = pd.DataFrame(details)

In [ ]:
len(P5)

In [ ]:
def current_time_min():
    time_in_mins = time.time()/60.0
    return round(time_in_mins, 4)



In [ ]:
import nltk
nltk.download('punkt')
import operator

In [ ]:
from summarizer.sbert import SBertSummarizer
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

#abs_summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)
abs_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
ext_model = SBertSummarizer('paraphrase-MiniLM-L6-v2')

In [ ]:
def summary_ners(article, entities, max_sentence_percent):

    #print(entities)
    entities_array = entities.split()
    # returns a string with ner sentences

    sentences_array = []
    ner_sentences = []

    sentences = nltk.sent_tokenize(article)
    #print(sentences)
    sentence_dict = {}
    sentence_order_dict = {}
    for k, sentence in enumerate(sentences):
        sentence_order_dict[sentence] = k
    
    sentence_count = len(sentences)
    #print("sentence count:", sentence_count)
    
    max_sentence_count = int(sentence_count * max_sentence_percent)
    #for sentence in sentences:
    for k, sentence in enumerate(sentences): 
        count = 0
        for word in entities_array:
            if word in sentence:
                count += 1
        #print(count)
        sentence_dict[sentence] = count
    
    sorted_sd = sorted(sentence_dict.items(), key=operator.itemgetter(1), reverse=True)

    for item in sorted_sd:
        ner_sentences.append(''.join(item[0]))
        #print(item[0], item[1])
    
    sentence_dict = {}
                
    for sentence in ner_sentences[:max_sentence_count]:
        sentence_dict[sentence] = sentence_order_dict[sentence]
        
    sorted_sd = sorted(sentence_dict.items(), key=operator.itemgetter(1), reverse=False)

    ner_sentences = []
    for item in sorted_sd:
        ner_sentences.append(''.join(item[0]))

    ner_sentences = ' '.join(ner_sentences)
            
    if len(ner_sentences) == 0:
        for item in sentences:
            len_sentences = min(max_sentence_count, len(sentences))
            ner_sentences = ' '.join(sentences[:len_sentences])
    
    return ner_sentences

In [ ]:
# article = "On New Year Day pioneering pilots celebrated float 125th Tournament Roses parade Pasadena. California part effort bring attention often overlooked part American history. Women pilots honored 65 years World War II service First lady Eleanor Roosevelt early supporter program writing September 1942 newspaper column This time women patient .  We war need fight ability every weapon possible .  Women pilots particular case weapon waiting used. Mary Matalin: The book good read unusually detailed frontrow seat view strained campaigns always are compelling insight Palin perspective. But impact personal professional relationships sad one indeed one. I hope conservatives let divide us marching toward promising midterm reflects ascendant commonsense conservatism requires good guys foxhole together. Cameron Todd Willingham executed 2004 13 years fire killed three daughters. Prosecutors argued Willingham deliberately set 1991 blaze  three reviews evidence outside experts found fire ruled arson. Texas Forensic Science Commission looking Willingham execution since 2008."
# entities = "Pasadena American Eleanor Roosevelt Willingham Palin Mary"
# summary_ner = summary_ners(article, entities, 0.3)
# #summary_ner
# #sentences = nltk.sent_tokenize(summary_ner)
# #print(len (sentences))
# print(summary_ner)

In [ ]:
def summary_ners_original(article, entities, max_sentence_count):

    # max_sentence_count = 50% & order retained
    
    print(entities)
    entities_array = entities.split()
    # returns a string with ner sentences

    sentences_array = []
    ner_sentences = []

    sentences = nltk.sent_tokenize(article)
    #print(sentences)
    sentence_dict = {}

    for sentence in sentences:
        count = 0
        for word in entities_array:
            if word in sentence:
                count += 1
        #print(count)
        sentence_dict[sentence] = count
    
    sorted_sd = sorted(sentence_dict.items(), key=operator.itemgetter(1), reverse=True)

    for item in sorted_sd:
        ner_sentences.append(''.join(item[0]))

    ner_sentences = ' '.join(ner_sentences[:max_sentence_count])
    
    if len(ner_sentences) == 0:
        for item in sentences:
            len_sentences = min(max_sentence_count, len(sentences))
            ner_sentences = ' '.join(sentences[:len_sentences])
    
    return ner_sentences

In [ ]:
# # https://pypi.org/project/bert-extractive-summarizer/
# # https://arxiv.org/abs/1908.10084

df = df_test

#abs_summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)
#abs_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
ext_model = SBertSummarizer('paraphrase-MiniLM-L6-v2')

def summarize_text(row):
    
    text = row[1]['text']
    headlines = row[1]['headlines']
    keywords = row[1]['keywords']
    entities = row[1]['entities']
    sentence_1s = row[1]['sentence_1s']
    sentence_3s = row[1]['sentence_3s']
   
    summary_art = row[1]['summary_art']
    summary_ext = row[1]['summary_ext']
    summary_abs = row[1]['summary_abs']
    summary_extabs = row[1]['summary_extabs']
    summary_ner = row[1]['summary_ner']
    summary_t5 = row[1]['summary_t5']
    max_len = min(len(summary_abs), 280)
    max_sentence_percent = 0.5
    
    try:
        #ext_model = SBertSummarizer('paraphrase-MiniLM-L6-v2')
        result = ext_model(text, num_sentences=10)
        summary_ext = ''.join(result)
        summary_extabs = abs_summarizer(summary_ext, max_length=min(len(summary_ext), 280), min_length=15, do_sample=False)[0]['summary_text']
        #summary_ner = summary_ners(text, entities, max_sentence_percent)
        summary_abs = abs_summarizer(text, max_length=min(len(text), 280), min_length=15, do_sample=False)[0]['summary_text']
    except Exception as error:
        summary_abs = abs_summarizer(text[:2048], max_length=min(len(text), 280), min_length=15, do_sample=False)[0]['summary_text']
        return text, headlines, keywords, entities, sentence_1s, sentence_3s, summary_art, summary_ext, summary_abs,  summary_extabs, summary_ner, summary_t5
    return text, headlines, keywords, entities, sentence_1s, sentence_3s, summary_art, summary_ext, summary_abs,  summary_extabs, summary_ner, summary_t5

In [ ]:

text = []
headlines = []
keywords = []
entities = []
sentence_1s = []
sentence_3s = []
summary_art = []
summary_ext = []
summary_abs = []
summary_extabs = []
summary_ner = []
summary_t5 = []

from joblib import Parallel, delayed
start = time.time()

df = df.reset_index()

S = Parallel(n_jobs=4)(delayed(summarize_text)(row) for row in df.iterrows())

print("Response time (mins): ", round((time.time() - start)/60, 2))

In [ ]:
S = list(filter(None, S))
for i in range(len(S)):  
        text.append(S[i][0])  # text
        headlines.append(S[i][1])  # headlines
        keywords.append(S[i][2])  # keywords
        entities.append(S[i][3])  # entities
        sentence_1s.append(S[i][4])  # sentence_1s
        sentence_3s.append(S[i][5])  # sentence_3s
        summary_art.append(S[i][6])  # summary_art
        summary_ext.append(S[i][7])  # summary_ext
        summary_abs.append(S[i][8])  # summary_abs
        summary_extabs.append(S[i][9])  # summary_extabs
        summary_ner.append(S[i][10])  # summary_ner
        summary_t5.append(S[i][11])  # summary_t5
        

In [ ]:
# summary_ner[0]
# sentences = nltk.sent_tokenize(summary_ner[0])
# print(len (sentences))
# print(summary_ner[0])

In [ ]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [ ]:
text = "On New Year Day pioneering pilots celebrated float 125th Tournament Roses parade Pasadena California part effort bring attention often overlooked part American history. Women pilots honored 65 years World War II service First lady Eleanor Roosevelt early supporter program writing September 1942 newspaper column This time women patient .  We war need fight ability every weapon possible .  Women pilots particular case weapon waiting used., On New Year Day pioneering pilots celebrated float 125th Tournament Roses parade Pasadena California part effort bring attention often overlooked part American history. Women pilots honored 65 years World War II service First lady Eleanor Roosevelt early supporter program writing September 1942 newspaper column This time women patient .  We war need fight ability every weapon possible .  Women pilots particular case weapon waiting used., On New Year Day pioneering pilots celebrated float 125th Tournament Roses parade Pasadena California part effort bring attention often overlooked part American history. Women pilots honored 65 years World War II service First lady Eleanor Roosevelt early supporter program writing September 1942 newspaper column This time women patient .  We war need fight ability every weapon possible .  Women pilots particular case weapon waiting used. On New Year Day pioneering pilots celebrated float 125th Tournament Roses parade Pasadena California part effort bring attention often overlooked part American history. Women pilots honored 65 years World War II service First lady Eleanor Roosevelt early supporter program writing September 1942 newspaper column This time women patient .  We war need fight ability every weapon possible .  Women pilots particular case weapon waiting used. On New Year Day pioneering pilots celebrated float 125th Tournament Roses parade Pasadena California part effort bring attention often overlooked part American history. Women pilots honored 65 years World War II service First lady Eleanor Roosevelt early supporter program writing September 1942 newspaper column This time women patient .  We war need fight ability every weapon possible .  Women pilots particular case weapon waiting used."

In [ ]:
# preprocess_text = text.strip().replace("\n","")
# t5_prepared_Text = "summarize: " + preprocess_text
# print ("original text preprocessed: \n", preprocess_text)

# tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


# # summmarize 
# summary_ids = model.generate(tokenized_text,
#                                     num_beams=4,
#                                     no_repeat_ngram_size=2,
#                                     min_length=30,
#                                     max_length=100,
#                                     early_stopping=True)

# output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# print ("\n\nSummarized text: \n",output)

In [ ]:


df = df_test

def summarize_text_t5(row):
    
    text = row[1]['text']
    #.split(' ', 512)
    #text = ' '.join([str(item) for item in text])
    max_len = min(len(text), 280)
    max_sentence_count = 5
    summary_t5 = []
    
    try:
        t5_prepared_Text = "summarize: " + text
        tokenized_text = tokenizer.encode(t5_prepared_Text, max_length=2048, return_tensors="pt").to(device) #default 512
        summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=max_len,
                                    early_stopping=True)

        summary_t5 = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        
    except Exception as error:
        t5_prepared_Text = "summarize: " + text
        tokenized_text = tokenizer.encode(t5_prepared_Text, max_length=6192, return_tensors="pt").to(device) #default 512
        summary_ids = model.generate(tokenized_text,
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=max_len,
                                    early_stopping=True)

        summary_t5 = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return text, summary_t5
    return text, summary_t5

In [ ]:

# text = []
# headlines = []
# keywords = []
# entities = []
# sentence_1s = []
# sentence_3s = []
# summary_art = []
# summary_ext = []
# summary_abs = []
# summary_extabs = []
# summary_ner = []
summary_t5 = []

from joblib import Parallel, delayed
start = time.time()

df = df.reset_index()

S = Parallel(n_jobs=4)(delayed(summarize_text_t5)(row) for row in df.iterrows())

print("Response time (mins): ", round((time.time() - start)/60, 2))

In [ ]:
summary_t5 = []
text = []
S = list(filter(None, S))
for i in range(len(S)):  
        text.append(S[i][0])  # text
        summary_t5.append(S[i][1])  # summary_t5

In [ ]:

# X = details_dict['text']
# Y = details_dict['headlines']
# Z = details_dict['keywords']
# S1 = details_dict['summary_art']
# S2 = details_dict['summary_ext']
# S3 = details_dict['summary_abs']
# S4 = details_dict['summary_extabs']
# S5 = details_dict['summary_ner']
# P1 = details_dict['predict_base']
# P2 = details_dict['predict_ext']
# P3 = details_dict['predict_abs']
# P4 = details_dict['predict_extabs']
# P5 = details_dict['predict_ner']


#X = details_dict['text']
#Y = details_dict['headlines']
#Z = details_dict['keywords']
# S1 = W
# S2 = V
# S3 = U
# S4 = [''] * size
#S5 = [''] * size

P0 = [''] * size
P1 = [''] * size
P2 = [''] * size
P3 = [''] * size
P4 = [''] * size
P5 = [''] * size
P6 = [''] * size
P7 = [''] * size

details = {
    'headlines' : headlines,
    'text' : text,
    'keywords' : keywords,
    'summary_art' : summary_art,
    'entities' : entities,
    'text_ner' : text_ner,
    'sentence_1s' : sentence_1s,
    'sentence_3s' : sentence_3s,
    'summary_ext' : summary_ext,
    'summary_abs' : summary_abs,
    'summary_extabs' : summary_extabs,
    'summary_ner' : summary_ner,
    'summary_t5' : summary_t5,
    'predict_1s' : P0,
    'predict_3s' : P1,
    'predict_text' : P2,
    'predict_ext' : P3,
    'predict_abs' : P4,
    'predict_extabs' : P5,
    'predict_ner' : P6,
    'predict_t5' : P7
}


# details = {
#     'headlines' : Y,
#     'text' : X,
#     'keywords' : Z,
#     'summary_art' : S1,
#     'entities' : E,
#     'sentence_1s' : S1s,
#     'sentence_3s' : S3s,
#     'summary_ext' : S2,
#     'summary_abs' : S3,
#     'summary_extabs' : S4,
#     'summary_ner' : S5,
#     'summary_t5' : summary_t5,
#     'predict_1s' : P0,
#     'predict_3s' : P1,
#     'predict_text' : P2,
#     'predict_ext' : P3,
#     'predict_abs' : P4,
#     'predict_extabs' : P5,
#     'predict_ner' : P6,
#     'predict_t5' : P7
# }


# details = {
#     'text' : text,
#     'headlines' : headlines,
#     'keywords' : keywords,
#     'summary_art' : summary_art,
#     'summary_ext' : summary_ext,
#     'summary_abs' : summary_abs,
#     'summary_extabs' : summary_extabs,
#     'summary_ner' : S5,
#     'predict_base' : P1,
#     'predict_ext' : P2,
#     'predict_abs' : P3,
#     'predict_extabs' : P4,
#     'predict_ner' : P5
# }

#df_test = pd.DataFrame(details)

jsonString = json.dumps(details)
jsonFile = open(os.path.join(path_processed_data, 'cnn_test_summary_v3.json'), "w")
jsonFile.write(jsonString)
jsonFile.close()


In [ ]:
#df_results = {k: details[k] for k in ('headlines', 'summary_art', 'summary_ext', 'summary_abs')}
df_results = {k: details[k] for k in ('summary_ext', 'summary_abs', 'summary_extabs', 'summary_ner', 'summary_t5')}
df_results = pd.DataFrame(df_results)
display(HTML(df_results.to_html()))

In [ ]:
#df_results = {k: details[k] for k in ('headlines', 'summary_art', 'summary_ext', 'summary_abs')}
df_results = {k: details[k] for k in ('sentence_1s', 'sentence_3s', 'text')}
df_results = pd.DataFrame(df_results)
display(HTML(df_results.to_html()))

In [ ]:
jsonFile = open(os.path.join(path_processed_data, input_filename), "r")
jsonContent = jsonFile.read()
details_dict = json.loads(jsonContent)
jsonFile.close()


X = details_dict['text']
size = len(X)

Y = details_dict['headlines']
Z = details_dict['keywords']
E = details_dict['entities']
text_ner = details_dict['text_ner']
S1s = details_dict['sentence_1s']
S3s = details_dict['sentence_3s']
S1 = details_dict['summary_art']
S2 = details_dict['summary_ext']
S3 = details_dict['summary_abs']
S4 = details_dict['summary_extabs']
S5 = details_dict['summary_ner']
#S6 = details_dict['summary_t5']
S6 = ['']*size

P0 = details_dict['predict_1s']
P1 = details_dict['predict_3s']
P2 = details_dict['predict_text']
P3 = details_dict['predict_ext']
P4 = details_dict['predict_abs']
P5 = details_dict['predict_extabs']
P6 = details_dict['predict_ner']
P7 = details_dict['predict_t5']


details = {
    
    'text' : X,
    'headlines' : Y,
    'keywords' : Z,
    'summary_art' : S1,
    'entities' : E,
    'text_ner' : text_ner,
    'sentence_1s' : S1s,
    'sentence_3s' : S3s,
    'summary_ext' : S2,
    'summary_abs' : S3,
    'summary_extabs' : S4,
    'summary_ner' : S5,
    'summary_t5' : S6,
    'predict_1s' : P0,
    'predict_3s' : P1,
    'predict_text' : P2,
    'predict_ext' : P3,
    'predict_abs' : P4,
    'predict_extabs' : P5,
    'predict_ner' : P6,
    'predict_t5' : P7
}



# size = len(X)

# S2 = [""] * size
# S3 = [""] * size
# S4 = [""] * size
# S5 = [""] * size
# P1 = [""] * size
# P2 = [""] * size
# P3 = [""] * size
# P4 = [""] * size
# P5 = [""] * size

# details = {
#     'text' : X,
#     'headlines' : Y,
#     'keywords' : Z,
#     'summary_art' : S1,
#     'summary_ext' : S2,
#     'summary_abs' : S3,
#     'summary_extabs' : S4,
#     'summary_ner' : S5,
#     'predict_text' : P1,
#     'predict_ext' : P2,
#     'predict_abs' : P3,
#     'predict_extabs' : P4,
#     'predict_ner' : P5
# }

df_test = pd.DataFrame(details)

In [ ]:
# # https://pypi.org/project/bert-extractive-summarizer/
# # https://arxiv.org/abs/1908.10084

df = df_test

#abs_summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)
#abs_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
ext_model = SBertSummarizer('paraphrase-MiniLM-L6-v2')

def summarize_text(row):
    
    text = row[1]['text_ner']
    headlines = row[1]['headlines']
    keywords = row[1]['keywords']
    entities = row[1]['entities']
    sentence_1s = row[1]['sentence_1s']
    sentence_3s = row[1]['sentence_3s']
   
    summary_art = row[1]['summary_art']
    summary_ext = row[1]['summary_ext']
    summary_abs = row[1]['summary_abs']
    summary_extabs = row[1]['summary_extabs']
    summary_ner = row[1]['summary_ner']
    summary_t5 = row[1]['summary_t5']
    max_len = min(len(summary_abs), 280)
    max_sentence_percent = 0.5
    
    try:
        #ext_model = SBertSummarizer('paraphrase-MiniLM-L6-v2')
        result = ext_model(text, num_sentences=10)
        summary_ext = ''.join(result)
        summary_extabs = abs_summarizer(summary_ext, max_length=min(len(summary_ext), 280), min_length=15, do_sample=False)[0]['summary_text']
        #summary_ner = summary_ners(text, entities, max_sentence_percent)
        summary_abs = abs_summarizer(text, max_length=min(len(text), 280), min_length=15, do_sample=False)[0]['summary_text']
    except Exception as error:
        summary_abs = abs_summarizer(text[:2048], max_length=min(len(text), 280), min_length=15, do_sample=False)[0]['summary_text']
        return text, headlines, keywords, entities, sentence_1s, sentence_3s, summary_art, summary_ext, summary_abs,  summary_extabs, summary_ner, summary_t5
    return text, headlines, keywords, entities, sentence_1s, sentence_3s, summary_art, summary_ext, summary_abs,  summary_extabs, summary_ner, summary_t5

In [ ]:
text_ner = []
headlines = []
keywords = []
entities = []
sentence_1s = []
sentence_3s = []
summary_art = []
summary_ext = []
summary_abs = []
summary_extabs = []
summary_ner = []
summary_t5 = []

from joblib import Parallel, delayed
start = time.time()

df = df.reset_index()

S = Parallel(n_jobs=4)(delayed(summarize_text)(row) for row in df.iterrows())

print("Response time (mins): ", round((time.time() - start)/60, 2))


In [ ]:
S = list(filter(None, S))
for i in range(len(S)):  
        text_ner.append(S[i][0])  # text
        headlines.append(S[i][1])  # headlines
        keywords.append(S[i][2])  # keywords
        entities.append(S[i][3])  # entities
        sentence_1s.append(S[i][4])  # sentence_1s
        sentence_3s.append(S[i][5])  # sentence_3s
        summary_art.append(S[i][6])  # summary_art
        summary_ext.append(S[i][7])  # summary_ext
        summary_abs.append(S[i][8])  # summary_abs
        summary_extabs.append(S[i][9])  # summary_extabs
        summary_ner.append(S[i][10])  # summary_ner
        summary_t5.append(S[i][11])  # summary_t5

In [ ]:

# X = details_dict['text']
# Y = details_dict['headlines']
# Z = details_dict['keywords']
# S1 = details_dict['summary_art']
# S2 = details_dict['summary_ext']
# S3 = details_dict['summary_abs']
# S4 = details_dict['summary_extabs']
# S5 = details_dict['summary_ner']
# P1 = details_dict['predict_base']
# P2 = details_dict['predict_ext']
# P3 = details_dict['predict_abs']
# P4 = details_dict['predict_extabs']
# P5 = details_dict['predict_ner']


#X = details_dict['text']
#Y = details_dict['headlines']
#Z = details_dict['keywords']
# S1 = W
# S2 = V
# S3 = U
# S4 = [''] * size
#S5 = [''] * size

P0 = [''] * size
P1 = [''] * size
P2 = [''] * size
P3 = [''] * size
P4 = [''] * size
P5 = [''] * size
P6 = [''] * size
P7 = [''] * size

details = {
    'headlines' : headlines,
    'text' : text,
    'keywords' : keywords,
    'summary_art' : summary_art,
    'entities' : entities,
    'text_ner' : text_ner,
    'sentence_1s' : sentence_1s,
    'sentence_3s' : sentence_3s,
    'summary_ext' : summary_ext,
    'summary_abs' : summary_abs,
    'summary_extabs' : summary_extabs,
    'summary_ner' : summary_ner,
    'summary_t5' : summary_t5,
    'predict_1s' : P0,
    'predict_3s' : P1,
    'predict_text' : P2,
    'predict_ext' : P3,
    'predict_abs' : P4,
    'predict_extabs' : P5,
    'predict_ner' : P6,
    'predict_t5' : P7
}


# details = {
#     'headlines' : Y,
#     'text' : X,
#     'keywords' : Z,
#     'summary_art' : S1,
#     'entities' : E,
#     'sentence_1s' : S1s,
#     'sentence_3s' : S3s,
#     'summary_ext' : S2,
#     'summary_abs' : S3,
#     'summary_extabs' : S4,
#     'summary_ner' : S5,
#     'summary_t5' : summary_t5,
#     'predict_1s' : P0,
#     'predict_3s' : P1,
#     'predict_text' : P2,
#     'predict_ext' : P3,
#     'predict_abs' : P4,
#     'predict_extabs' : P5,
#     'predict_ner' : P6,
#     'predict_t5' : P7
# }


# details = {
#     'text' : text,
#     'headlines' : headlines,
#     'keywords' : keywords,
#     'summary_art' : summary_art,
#     'summary_ext' : summary_ext,
#     'summary_abs' : summary_abs,
#     'summary_extabs' : summary_extabs,
#     'summary_ner' : S5,
#     'predict_base' : P1,
#     'predict_ext' : P2,
#     'predict_abs' : P3,
#     'predict_extabs' : P4,
#     'predict_ner' : P5
# }

#df_test = pd.DataFrame(details)

jsonString = json.dumps(details)
jsonFile = open(os.path.join(path_processed_data, 'cnn_test_summary_ner_v3.json'), "w")
jsonFile.write(jsonString)
jsonFile.close()


In [ ]:
#df_results = {k: details[k] for k in ('headlines', 'summary_art', 'summary_ext', 'summary_abs')}
df_results = {k: details[k] for k in ('summary_ext', 'summary_abs', 'summary_extabs', 'summary_art', 'summary_t5')}
df_results = pd.DataFrame(df_results)
display(HTML(df_results.to_html()))

In [ ]:
#df_results = {k: details[k] for k in ('headlines', 'summary_art', 'summary_ext', 'summary_abs')}
df_results = {k: details[k] for k in ('sentence_1s', 'sentence_3s', 'text')}
df_results = pd.DataFrame(df_results)
display(HTML(df_results.to_html()))